# Installing Librarys

In [27]:
%pip install neo4j
from neo4j import GraphDatabase, exceptions

# Setting up driver

In [40]:
driver = GraphDatabase.driver("neo4j+s://d0eb4ffb.databases.neo4j.io", auth=("neo4j", "tuU6ZWnGsMTth1bQmz5iFT3mmhgZJNOnsjoPM5GbQAw"))

# Stärke 1

In [29]:
def create_relationship(tx, name1, name2):
    tx.run("""
    CREATE (a:Person {name: $name1})
    RETURN a
    """, name1=name1)
    result = tx.run("""
    MATCH (a:Person {name: $name1})
    MATCH (b:Person {name: $name2})
    MERGE (a)-[:KNOWS]->(b)
    RETURN a, b
    """, name1=name1, name2=name2)
    if not result.single():
        raise Exception("Kann Beziehung nicht erstellen")

def transaction_example(driver):
    with driver.session() as session:
        try:
            session.write_transaction(create_relationship, "Test_User", "Nicko")
        except Exception as e:
            print("Transaktion fehlgeschlagen: ", e)

transaction_example(driver)



C:\Users\David Dedic\AppData\Local\Temp\ipykernel_17696\831392994.py:18: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_relationship, "Test_User", "Nicko")


Transaktion fehlgeschlagen:  Kann Beziehung nicht erstellen


In [31]:
def find_person(tx, name):
    result = tx.run("MATCH (p:Person) WHERE p.name = $name RETURN p.name AS name", name=name)
    record = result.single() # fetch only one record
    if record is None:
        print(f"Person mit dem Namen '{name}' existiert nicht.")
    else:
        print(record["name"])
with driver.session() as session:
    session.read_transaction(find_person, "Test_User")


Person mit dem Namen 'Test_User' existiert nicht.


C:\Users\David Dedic\AppData\Local\Temp\ipykernel_17696\687030250.py:9: DeprecationWarning: read_transaction has been renamed to execute_read
  session.read_transaction(find_person, "Test_User")


In [24]:
find_person(driver.session(), "Test_User")

# Stärke 2

In [42]:
def get_shortest_path(driver):
    with driver.session() as session:
        result = session.run("MATCH (p1:Person {name: 'Tom Hanks'}), (p2:Person {name: 'Steve Zahn'}) "
                             "MATCH path = shortestPath((p1)-[*]-(p2)) "
                             "RETURN path")
        return result.single()

path = get_shortest_path(driver)
print(path)

<Record path=<Path start=<Node element_id='4:a073acd0-a5a2-4eaf-a569-5c35c2336960:64' labels=frozenset({'Person'}) properties={'born': 1956, 'name': 'Tom Hanks'}> end=<Node element_id='4:a073acd0-a5a2-4eaf-a569-5c35c2336960:39' labels=frozenset({'Person'}) properties={'born': 1967, 'name': 'Steve Zahn'}> size=2>>
